What about calculating price per acre?

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from kaggle.data import load_data, save_submission
from kaggle.metrics import rmse
from kaggle.blender import train_regressors, blend_predictions

In [3]:
y, X, X_submission, ids_submission = load_data()
y = np.log(y)

In [46]:
regressors1 = [RandomForestRegressor(n_estimators=100, random_state=1773),
               ExtraTreesRegressor(n_estimators=100, random_state=1773),
               GradientBoostingRegressor(n_estimators=50, random_state=1773)]

In [47]:
regressors2 = [RandomForestRegressor(n_estimators=100, random_state=1773),
               ExtraTreesRegressor(n_estimators=100, random_state=1773),
               GradientBoostingRegressor(n_estimators=50, max_depth=5, min_samples_leaf=4, random_state=1773)]

In [35]:
kf = KFold(n_splits=10, random_state=1773)

In [36]:
oof_train, oof_test = train_regressors(X, X_submission, y, kf, regressors1)

In [43]:
for i in range(len(regressors)):
    print rmse(y, oof_train[:, i])

0.143096331205
0.139874686479
0.130707662451


In [44]:
blend_train, blend_test = blend_predictions(oof_train, oof_test, y, kf, rmse)

weights: [ -3.26746911e-22   2.79789412e-01   7.20210588e-01]


In [40]:
rmse(blend_train, y)

0.12901007964319375

In [41]:
X_stacked = np.hstack((X, oof_train))
X_submission_stacked = np.hstack((X_submission, oof_test))

In [42]:
oof_train_stacked, oof_test_stacked = train_regressors(X_stacked, X_submission_stacked, y, kf, regressors2)

In [45]:
for i in range(len(regressors)):
    print rmse(y, oof_train_stacked[:, i])

0.135923934153
0.13580233321
0.136930393086


In [31]:
blend_train_stacked, blend_test_stacked = blend_predictions(oof_train_stacked, oof_test_stacked, y, kf, rmse)

weights: [ 0.07947667  0.30497199  0.61555134]


In [32]:
rmse(blend_train_stacked, y)

0.13278533210050902

In [ ]:
y_pred = model.predict(X_submission)
save_submission(ids_submission, y_pred, '0_random_search_shifted_xgboost')